In [1]:
import csv
import spacy
from spacy import displacy
import numpy as np
import pandas as pd
import string
from flask import Flask
from flask import render_template
import io

In [2]:
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
from flask import Flask, redirect, url_for, render_template, request
app = Flask(__name__)

@app.route('/fun')
def index():
    return render_template('home.html')

data_good_review = pd.read_csv('Rated 5 out of 5.csv')
data_bad_review = pd.read_csv('Rated 1 out of 5.csv')
nlp=spacy.load('en_core_web_sm')

dataset = data_good_review.append([data_bad_review], ignore_index=True)
dataset.shape

punct = string.punctuation

stopwards=list(STOP_WORDS)

def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_ 
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwards and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

X = dataset['feedback']
y = dataset['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

@app.route('/', methods=['POST','GET'])
def get_output():
    data_request = request.form
    
    review = request.form['feedback']

    output=clf.predict([review])
    output1=output
    print(output1)
    if output1=='Rated 5 out of 5':
        print('Thank you for your feedback. ')
    else:
        print('Thank you for your feedback, We will make service better.')

    return render_template('render.html', output1=output1)


if __name__=="__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Dec/2022 20:58:29] "GET / HTTP/1.1" 400 -
127.0.0.1 - - [31/Dec/2022 20:58:36] "GET /fun HTTP/1.1" 200 -


['Rated 5 out of 5']
Thank you for your feedback. 


127.0.0.1 - - [31/Dec/2022 20:59:04] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2022 20:59:04] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [31/Dec/2022 20:59:24] "GET /fun HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2022 20:59:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2022 20:59:41] "GET /static/style.css HTTP/1.1" 404 -


['Rated 1 out of 5']
Thank you for your feedback, We will make service better.
